In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn import preprocessing
import time
from datetime import datetime

In [ ]:
# Combine all covid csv files together 
path = r'/Users/violetye/Desktop/csse_covid_19_daily_reports_us' 
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

covid = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
covid = covid.sort_values(by=['Province_State','Last_Update'], ignore_index = True)

In [ ]:
covid.head()

,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,People_Tested,Mortality_Rate
0,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,NaN,NaN,1.0,74.788122,NaN,437.0,NaN,84000001.0,USA,1265.075660,12.264945,62029.0,2.536133
1,Alabama,US,2020-04-13 23:07:54,32.3182,-86.9023,3870,99,NaN,NaN,1.0,78.928288,NaN,457.0,NaN,84000001.0,USA,1318.245181,12.238886,64636.0,2.558140
2,Alabama,US,2020-04-14 23:33:31,32.3182,-86.9023,4041,114,NaN,NaN,1.0,82.415817,NaN,493.0,NaN,84000001.0,USA,1377.818704,12.471541,67557.0,2.821084
3,Alabama,US,2020-04-15 22:56:51,32.3182,-86.9023,4307,118,NaN,NaN,1.0,87.840863,NaN,525.0,NaN,84000001.0,USA,1444.428468,12.883436,70823.0,2.739726
4,Alabama,US,2020-04-16 23:30:51,32.3182,-86.9023,4465,133,NaN,NaN,1.0,91.063258,NaN,553.0,NaN,84000001.0,USA,1506.734908,12.727273,73878.0,2.978723


In [ ]:
len(covid)

41314

In [ ]:
covid.Province_State.unique()

array(['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'Diamond Princess', 'District of Columbia', 'Florida', 'Georgia',
       'Grand Princess', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Puerto Rico', 'Recovered',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virgin Islands', 'Virginia',
       'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [ ]:
covid_df = covid[['Province_State','Last_Update','Confirmed','Deaths','Recovered']]
covid_df = covid_df.rename(columns={"Last_Update": "Covid_Date"})

In [ ]:
covid_df.head()

,Province_State,Covid_Date,Confirmed,Deaths,Recovered
0,Alabama,2020-04-12 23:18:15,3667,93,NaN
1,Alabama,2020-04-13 23:07:54,3870,99,NaN
2,Alabama,2020-04-14 23:33:31,4041,114,NaN
3,Alabama,2020-04-15 22:56:51,4307,118,NaN
4,Alabama,2020-04-16 23:30:51,4465,133,NaN


In [ ]:
# Get population for each state from the U.S.Census Bureau website
# https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html#par_textimage_1873399417
!curl https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv > state_pop.csv
state_pop = pd.read_csv('state_pop.csv')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 74085    0 74085    0     0   648k      0 --:--:-- --:--:-- --:--:--     0--:-- --:--:-- --:--:--  651k


In [ ]:
state_pop = state_pop[['NAME','POPESTIMATE2019']]
state_pop = state_pop.rename(columns={"NAME": "Province_State", "POPESTIMATE2019": "Pop"})

In [ ]:
state_pop.Province_State.unique()

array(['United States', 'Northeast Region', 'Midwest Region',
       'South Region', 'West Region', 'Alabama', 'Alaska', 'Arizona',
       'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [ ]:
covid_df2 = pd.merge(covid_df,
                 state_pop,
                 on='Province_State')

In [ ]:
covid_df2.head()

,Province_State,Covid_Date,Confirmed,Deaths,Recovered,Pop
0,Alabama,2020-04-12 23:18:15,3667,93,NaN,4903185
1,Alabama,2020-04-13 23:07:54,3870,99,NaN,4903185
2,Alabama,2020-04-14 23:33:31,4041,114,NaN,4903185
3,Alabama,2020-04-15 22:56:51,4307,118,NaN,4903185
4,Alabama,2020-04-16 23:30:51,4465,133,NaN,4903185


In [ ]:
# Basic clean-up and dates features
le = preprocessing.LabelEncoder()
covid_df_corr = covid_df2.copy()
covid_df_corr['Covid_Date'] = pd.to_datetime(covid_df_corr['Covid_Date'])
covid_df_corr['Day_Num'] = le.fit_transform(covid_df_corr.Covid_Date)
covid_df_corr['Day'] = covid_df_corr['Covid_Date'].dt.day
covid_df_corr['Month'] = covid_df_corr['Covid_Date'].dt.month
covid_df_corr['Year'] = covid_df_corr['Covid_Date'].dt.year

In [ ]:
# Fill null values given that we merged train-test datasets
covid_df_corr['Province_State'].fillna("None", inplace=True)
covid_df_corr['Confirmed'].fillna(0, inplace=True)
covid_df_corr['Deaths'].fillna(0, inplace=True)
covid_df_corr['Recovered'].fillna(0, inplace=True)

In [ ]:
# lag calculation
def calculate_lag(df, lag_list, column):
    for lag in lag_list:
        column_lag = "Lag_" + column + "_" + str(lag)
        df[column_lag] = df[column].shift(lag, fill_value=0)
    return df

In [ ]:
# trend calculation
def calculate_trend(df, lag_list, column):
    for lag in lag_list:
        trend_column_lag = "Trend_" + column + "_" + str(lag)
        df[trend_column_lag] = (df[column]-df[column].shift(lag, fill_value=-999))/df[column].shift(lag, fill_value=0)

        # df[col1] = df[column] + df[column2]
    return df

In [ ]:
ts = time.time()
covid_df_corr = calculate_lag(covid_df_corr, range(1,7), 'Confirmed')
covid_df_corr = calculate_lag(covid_df_corr, range(1,7), 'Deaths')
covid_df_corr = calculate_lag(covid_df_corr, range(1,7), 'Recovered')

In [ ]:
covid_df_corr = calculate_trend(covid_df_corr, range(1,7), 'Confirmed')
covid_df_corr = calculate_trend(covid_df_corr, range(1,7), 'Deaths')
covid_df_corr = calculate_trend(covid_df_corr, range(1,7), 'Recovered')

In [ ]:
covid_df_corr.replace([np.inf, -np.inf], 0, inplace=True)
covid_df_corr.fillna(0, inplace=True)

In [ ]:
covid_df_corr['Confirmed_ema'] = covid_df_corr['Confirmed'].ewm(span=7,adjust=False).mean()
covid_df_corr['Deaths_ema'] = covid_df_corr['Deaths'].ewm(span=7,adjust=False).mean()
covid_df_corr['Recovered_ema'] = covid_df_corr['Recovered'].ewm(span=7,adjust=False).mean()

In [ ]:
covid_df_corr.head()

,Province_State,Covid_Date,Confirmed,Deaths,Recovered,Pop,Day_Num,Day,Month,Year,...,Trend_Deaths_6,Trend_Recovered_1,Trend_Recovered_2,Trend_Recovered_3,Trend_Recovered_4,Trend_Recovered_5,Trend_Recovered_6,Confirmed_ema,Deaths_ema,Recovered_ema
0,Alabama,2020-04-12 23:18:15,3667,93,0.0,4903185,0,12,4,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3667.000000,93.000000,0.0
1,Alabama,2020-04-13 23:07:54,3870,99,0.0,4903185,1,13,4,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3717.750000,94.500000,0.0
2,Alabama,2020-04-14 23:33:31,4041,114,0.0,4903185,3,14,4,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3798.562500,99.375000,0.0
3,Alabama,2020-04-15 22:56:51,4307,118,0.0,4903185,4,15,4,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3925.671875,104.031250,0.0
4,Alabama,2020-04-16 23:30:51,4465,133,0.0,4903185,5,16,4,2020,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4060.503906,111.273438,0.0


In [ ]:
def add_extra_trends(df):

    df1 = df.copy()

    df1['NewConfirmed'] = df1['Confirmed'] - df1['Confirmed'].shift(1)
    df1['NewConfirmed'] = df1['NewConfirmed'].fillna(df1['Confirmed'])
    df1['PreviousDayNewConfirmed'] = df1['NewConfirmed']
    df1['PreviousDayNewConfirmed'] = df1['PreviousDayNewConfirmed'].fillna(0)
    df1['growth_factor'] = df1['Confirmed'] / df1['PreviousDayNewConfirmed'] # https://www.youtube.com/watch?v=Kas0tIxDvrg
    df1['growth_factor'] = df1['growth_factor'].rolling(3).median() # add smoothing
        

    df1['NewFatalities'] = df1['Deaths'] - df1['Deaths'].shift(1)
    df1['NewFatalities'] = df1['NewFatalities'].fillna(df1['Deaths'])
    df1['PreviousDayNewFatalities'] = df1['Deaths'].shift(1) - df1['Deaths'].shift(2)
    df1['PreviousDayNewFatalities'] = df1['PreviousDayNewFatalities'].fillna(0)
    df1['mortality_factor'] = df1['NewFatalities'] / df1['PreviousDayNewFatalities']
    df1['mortality_factor'] = df1['mortality_factor'].rolling(3).median() # add smoothing
    df1['case_fatality_rate'] = df1['Deaths'] / df1['Confirmed']
    df1['infection_rate'] = df1['Confirmed'] / df1['Pop']
    df1['mortality_rate'] = df1['Deaths'] / df1['Pop']
        
    df1 = df1.replace([np.inf, -np.inf], np.nan)
    df1['growth_factor'] = np.where(df1['Confirmed']==0, 0, df1['growth_factor'].fillna(1.25)) # assumed
    df1['mortality_factor'] = np.where(df1['Deaths']==0, 0, df1['mortality_factor'].fillna(1.25)) # assumed
    df1['case_fatality_rate'] = round(df1['case_fatality_rate'].fillna(0), 4)
    df1['infection_rate'] = round(df1['infection_rate'].fillna(0), 4)
    df1['mortality_rate'] = round(df1['mortality_rate'].fillna(0), 4)
      
    return df1.drop(['NewConfirmed', 'PreviousDayNewConfirmed', 'NewFatalities', 'PreviousDayNewFatalities'], axis=1)

covid_df_corr = add_extra_trends(covid_df_corr)
display(covid_df_corr.head(10))
    

,Province_State,Covid_Date,Confirmed,Deaths,Recovered,Pop,Day_Num,Day,Month,Year,...,Trend_Recovered_5,Trend_Recovered_6,Confirmed_ema,Deaths_ema,Recovered_ema,growth_factor,mortality_factor,case_fatality_rate,infection_rate,mortality_rate
0,Alabama,2020-04-12 23:18:15,3667,93,0.0,4903185,0,12,4,2020,...,0.0,0.0,3667.000000,93.000000,0.0,1.250000,1.25,0.0254,0.0007,0.0
1,Alabama,2020-04-13 23:07:54,3870,99,0.0,4903185,1,13,4,2020,...,0.0,0.0,3717.750000,94.500000,0.0,1.250000,1.25,0.0256,0.0008,0.0
2,Alabama,2020-04-14 23:33:31,4041,114,0.0,4903185,3,14,4,2020,...,0.0,0.0,3798.562500,99.375000,0.0,19.064039,1.25,0.0282,0.0008,0.0
3,Alabama,2020-04-15 22:56:51,4307,118,0.0,4903185,4,15,4,2020,...,0.0,0.0,3925.671875,104.031250,0.0,19.064039,1.25,0.0274,0.0009,0.0
4,Alabama,2020-04-16 23:30:51,4465,133,0.0,4903185,5,16,4,2020,...,0.0,0.0,4060.503906,111.273438,0.0,23.631579,2.50,0.0298,0.0009,0.0
5,Alabama,2020-04-17 23:30:52,4557,148,0.0,4903185,6,17,4,2020,...,0.0,0.0,4184.627930,120.455078,0.0,28.259494,1.00,0.0325,0.0009,0.0
6,Alabama,2020-04-18 22:32:47,4788,153,0.0,4903185,7,18,4,2020,...,0.0,0.0,4335.470947,128.591309,0.0,28.259494,1.00,0.0320,0.0010,0.0
7,Alabama,2020-04-19 23:41:01,4978,157,0.0,4903185,8,19,4,2020,...,0.0,0.0,4496.103210,135.693481,0.0,26.200000,0.80,0.0315,0.0010,0.0
8,Alabama,2020-04-20 23:36:47,5163,163,0.0,4903185,9,20,4,2020,...,0.0,0.0,4662.827408,142.520111,0.0,26.200000,0.80,0.0316,0.0011,0.0
9,Alabama,2020-04-21 23:40:34,5404,183,0.0,4903185,11,21,4,2020,...,0.0,0.0,4848.120556,152.640083,0.0,26.200000,1.50,0.0339,0.0011,0.0


In [ ]:
covid_df_corr.to_csv('covid_df_cleaned.csv')